# Downloading MODIS NDVI


## Load packages

In [1]:
import os
import ee
import geemap
import wxee
import warnings
import xarray as xr
import pandas as pd
import numpy as np
from odc.geo.xr import assign_crs

## Authenticate GEE    

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXnL2ffYUyHLy59BBTkZNtnuXQhhpyf5jqQNIRdFsGekj8kBsqTrk7w



Successfully saved authorization token.


## Analysis parameters


In [3]:
product = 'MODIS/MCD43A4_006_NDVI'
bands = ['NDVI']
start_date = '3/1/2000'
end_date = '12/31/2000'
output_crs = "EPSG:3577"
resolution = 5000

# ~Aus region (slightly less to trick google into giving me 1km res)
region = ee.Geometry.Polygon([[
            [114,-43.0],
            [154.0,-43.0],
            [154.0,-10.0],
            [113,-10.0],
            [113,-43.0]]])


In [11]:
#use start and end dates to loop through months and load GEE data
start = pd.date_range(start=start_date, end=end_date, freq='MS') 
end = pd.date_range(start=start_date, end=end_date, freq='M')

i = 0
for s, e in zip(start,end):
    print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

    ss = s+pd.Timedelta(14, 'd')

    s = s.strftime('%Y-%m-%d')
    e = e.strftime('%Y-%m-%d')

    ds = ee.ImageCollection(product) \
      .filterDate(s,e) \
      .map(lambda image: image.clip(region)) \
      .select(bands) \
      .reduce(ee.Reducer.median())  
 
    export = ee.batch.Export.image.toDrive(ds,
        description='MODIS_NDVI_median'+'_'+ss.strftime('%Y-%m-%d'),
        folder='MODIS_NDVI_median',
        region=region,
        scale=resolution,
        crs=output_crs,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized':True
              }
        )
    export.start()

    i+=1

In [12]:
export.status()

{'state': 'RUNNING',
 'description': 'MODIS_NDVI_median_2000-12-15',
 'creation_timestamp_ms': 1696803882552,
 'update_timestamp_ms': 1696803938667,
 'start_timestamp_ms': 1696803938633,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'VYEZXJKGQ436NWKZHFCBIWRF',
 'name': 'projects/earthengine-legacy/operations/VYEZXJKGQ436NWKZHFCBIWRF'}

In [ ]:
# Using basemap in geemap
Map = geemap.Map(center=[-35.2041, 149.2721], zoom=12)

Map.addLayer(ds)
Map

## Stitch together geotiffs

In [13]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from odc.geo.xr import assign_crs

In [14]:
base='/g/data/os22/chad_tmp/climate-carbon-interactions/data/MODIS_NDVI_median/'
name= 'NDVI_median'
dates_start = '2000-03-01'
dates_end = '2000-12-31'


In [15]:
files = [base+i for i in os.listdir(base) if i.endswith('.tif')]
files.sort()

time = (pd.date_range(dates_start, dates_end, freq='MS') + pd.DateOffset(days=14)).to_numpy()
time = list(time)

print(len(files), print(len(time)))

10
10 None


In [16]:
i=0
arrs=[]
for f, t in zip(files, time):
    print(" {:02}/{:02}\r".format(i + 1, len(files)), end="")
    
    ds = xr.open_dataarray(f).squeeze().drop('band').astype('float32')
    ds = ds.expand_dims(time=[t])
    ds = ds.rename(name)
    del ds.attrs['long_name']
    del ds.attrs['AREA_OR_POINT']

    ds.attrs['nodata']=np.nan
    ds = ds.drop('spatial_ref')
    ds = assign_crs(ds, crs='epsg:3577')
    arrs.append(ds)

    i+=1 
    

In [17]:
ds = xr.concat(arrs, dim='time').sortby('time')

In [30]:
mask = xr.open_dataarray('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/MODIS_NDVI_5km_monthly_2001_2022.nc').mean('time')
mask = ~np.isnan(mask)

In [33]:
ds = ds.where(mask)

In [37]:
ds.to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/MODIS_'+name+'_5km_monthly_2000.nc')

In [ ]:
# %%time
# #use start and end dates to loop through months and load GEE data

# start = pd.date_range(start=start_date, end=end_date, freq='MS') 
# end = pd.date_range(start=start_date, end=end_date, freq='M')

# i = 0
# for s, e in zip(start,end):
#     print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

#     ss = s+pd.Timedelta(14, 'd')

#     if os.path.isfile(output_path+'MODIS_NDVI_'+ss.strftime('%Y-%m')+'.nc'):
#         i+=1
#         pass

#     else:
#         s = s.strftime('%Y-%m-%d')
#         e = e.strftime('%Y-%m-%d')

#         ds = ee.ImageCollection(product) \
#           .filterDate(s,e) \
#           .map(lambda image: image.clip(region))

#         ds = ds.select(bands)

#         ts = wxee.TimeSeries(ds)
#         ds = ts.wx.to_xarray(region=region, scale=resolution, crs=output_crs, progress=False, num_cores=4)

#         warnings.filterwarnings("ignore")
#         ds = ds.astype(np.float32)

#         ds_mean = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()

#         ds_mean = ds_mean.rename({'NDVI':'NDVI_mean'})
#         ds_mean.attrs['nodata'] = np.nan

#         ds_out = assign_crs(ds_mean, crs='epsg:3577')

#         ds_out.to_netcdf(output_path+'MODIS_NDVI_'+ss.strftime('%Y-%m')+'.nc')

#         i+=1
